# Process fastq files


Install [`kb-python`](https://www.kallistobus.tools/kb_usage/kb_usage/#kallisto-and-bustools)

In [ ]:
%pip install kb-python


## Build a reference
Download reference files used by Cell Ranger, by uncommenting the corresponding "`species`" line. The references can also be directly downloaded from the [10x support page](https://support.10xgenomics.com/single-cell-gene-expression/software/downloads/latest).

NOTE: If the species are not human nor mouse, you should download the corresponding transcriptome reference, t2g and gtf files, from your favorite source and modify the paths accordingly.

In [ ]:
#species = "human"
species = "mouse"

if species == "human":
    # Human reference
    !curl -O https://cf.10xgenomics.com/supp/cell-exp/refdata-gex-GRCh38-2020-A.tar.gz
    !mkdir ./human-ref
    !tar -xvf refdata-gex-GRCh38-2020-A.tar.gz --directory ./human-ref
elif species == "mouse":
    # Mouse reference
    !curl -O https://cf.10xgenomics.com/supp/cell-exp/refdata-gex-mm10-2020-A.tar.gz
    !mkdir ./mouse-ref
    !tar -xvf refdata-gex-mm10-2020-A.tar.gz --directory ./mouse-ref

Build the reference:

In [ ]:
# specify directories/files
if species == "human":
    !ref_dir="./human-ref"
elif species == "mouse":
    !ref_dir="./mouse-ref"

!dna_fa="${ref_dir}/fasta/genome.fa"
!gtf="${ref_dir}/genes/genes.gtf"

# run kb ref
!kb ref -i ${ref_dir}transcriptome.idx -g ${ref_dir}t2g.txt -f1 ${ref_dir}cdna.fa $dna_fa $gtf

## Check 10x chemistry from fastq files

Download 10x barcode whitelists using `wget`. You can also directly download from 
the [cellranger github repository](https://github.com/10XGenomics/cellranger/tree/master/lib/python/cellranger/barcodes)

In [ ]:
!wget https://github.com/10XGenomics/cellranger/blob/master/lib/python/cellranger/barcodes/3M-february-2018.txt.gz
!gunzip ./3M-february-2018.txt.gz
!wget https://github.com/10XGenomics/cellranger/blob/master/lib/python/cellranger/barcodes/737K-april-2014_rc.txt
!wget https://github.com/10XGenomics/cellranger/blob/master/lib/python/cellranger/barcodes/737K-august-2016.txt

Check 10x chemistry using one of the fastq files, assuming that all fastq files were generated with the same chemistry

In [8]:
# specify fastq directory
from os import environ
fastq_dir = './fastq/'

environ['fastq_dir'] = fastq_dir

/Users/sara/Documents/Data/kaitilin/fastq/


Get the barcodes from any one of the R1 files. Replace the path with one of yours.

In [93]:
# get barcodes list from R1 file (assuming barcode length of 16 bases) 
!gzcat ./fastq/path-to-R1-file.fastq.gz | grep "@" -A 1 | grep -v "@" | grep -v "\-\-" | cut -c 1-16 > ./R1_bc

In [102]:
# read barcodes list and whitelists
import pandas as pd
from pathlib import Path

# path to the extracted R1 barcodes
fpath = Path(fastq_dir, "R1_bc")

# read extracted R1 barcodes
r1bc = pd.read_csv(fpath, header = None)

# load whitelists for all 10x chromium chemistries
wht_v3 = pd.read_csv('./3M-february-2018.txt', header = None)
wht_v2 = pd.read_csv('./737K-august-2016.txt', header = None) 
wht_v1 = pd.read_csv('./737K-april-2014_rc.txt', header = None) 

# calculate fractions of barcodes overlapping the whitelist for each specific chemistry
v3 = pd.merge(r1bc, wht_v3, how='inner', on=[0])
v2 = pd.merge(r1bc, wht_v2, how='inner', on=[0])
v1 = pd.merge(r1bc, wht_v1, how='inner', on=[0])

In [113]:
# get max value for fractions of overlapping barcodes
maxf = max(v3.size, v2.size, v1.size) 

In [ ]:
# identify 10x chemistry
if v3.size == maxf:
  print("10X chemistry is V3")
  tec = "10xv3"
elif v2.size == maxf:
  print("10X chemistry is V2")
  tec = "10xv2"
else:
  print("10X chemistry is V1")
  tec = "10xv1"

## Check species

Check species with FastQ Screen to align fastq files to the correct genome.
Download and install FastQ Screen: https://stevenwingett.github.io/FastQ-Screen/
Obtain pre-built Bowtie2 indices of commonly used reference genomes for FastQ Screen:

In [ ]:
!fastq_screen --get_genomes -outdir ./fastq_screen_genomes
!fastq_screen /path/to/some/R2/ -conf ./fastq_screen_genomes/fastq_screen.conf

# read FastQ Screen output
fsout = pd.read_csv('path/to/screen/output/', sep='\t', engine='python', skiprows=1)

# find min value of %Unmapped
unmapped = fsout["%Unmapped"]
min_index = unmapped.idxmin()

# get species
genome = fsout["Genome"]
fastq_screen_species = genome[min_index].lower()
print("Species is", fastq_screen_species)

## Generate a raw count matrix

To generate the count matrices for each sample, we have to run `kb count`. It needs several arguments:
1. `-i`: the index file, generated by `kb ref`. Its name should be "`species-reftranscriptome.idx`". Located in the "`species-ref`" directory (with the correct species)
2. `-g`: the gtf file. Its name should be "`species-reft2g.txt`"
3. `-x`: the 10x technology, which can be known or derived from the barcodes, as we did previously. Should be one of the strings `"10xv1"`, `"10xv2"`, `"10xv3"`. If inferred, should be saved in the variable "`$tec`"
4. `-o`: the output folder. Should be the name of the sample.
5. The fastq files. Should be an even number of files, and the same amount of R1's and R2's. You should add all R1's and R2's corresponding to the sample. separated by spaces (or \ to break the line). And they should be ordered R1 then R2, then R1... etc.

This needs doing for each sample. We leave two examples filled out, but they should be replaced by your own samples. We assume that the species is Mus musculus and that the 10x chemistry is version 3 ("10xv3"), and that we have 4 fastq files per sample

In [ ]:
# run kb count
!kb count -i ./mouse-ref/mouse-reftranscriptome.idx -g ./mouse-ref/mouse-reft2g.txt -x "10xv3" -o ./SAMPLE_1_kbcount_output \
./fastq/SAMPLE_1_L001_R1_001.fastq.gz \
./fastq/SAMPLE_1_L001_R2_001.fastq.gz \
./fastq/SAMPLE_1_L002_R1_001.fastq.gz \
./fastq/SAMPLE_1_L002_R2_001.fastq.gz \


!kb count -i ./mouse-ref/mouse-reftranscriptome.idx -g ./mouse-ref/mouse-reft2g.txt -x "10xv3" -o ./SAMPLE_2_kbcount_output \
./fastq/SAMPLE_2_L001_R1_001.fastq.gz \
./fastq/SAMPLE_2_L001_R2_001.fastq.gz \
./fastq/SAMPLE_2_L002_R1_001.fastq.gz \
./fastq/SAMPLE_2_L002_R2_001.fastq.gz \

# keep adding these lines for the extra samples. 


## Convert to 10x files

Lastly, we have to convert the kallisto bustools output to cellranger/10x like files (one folder per sample, each with barcodes/features/matrix files). For that we have an R script that does it automatically. You should have the reformat.R file inside the same folder as the notebook. Running the following cell should take care of things, but in case Rscript is not found, the simplest solution is to run the contents of the script interactively in Rstudio.

In [ ]:
!Rscript ./reformat.R $species